In [2]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D
import os
import numpy as np
import pandas as pd

In [12]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [13]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [14]:
from tqdm.notebook import tqdm

In [15]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels


In [18]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)



angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed


In [20]:
print(train)

                              image  label
0          images/train\angry\0.jpg  angry
1          images/train\angry\1.jpg  angry
2         images/train\angry\10.jpg  angry
3      images/train\angry\10002.jpg  angry
4      images/train\angry\10016.jpg  angry
...                             ...    ...
24463    images/train\sad\34643.jpg    sad
24464    images/train\sad\34653.jpg    sad
24465    images/train\sad\34661.jpg    sad
24466    images/train\sad\34666.jpg    sad
24467    images/train\sad\34672.jpg    sad

[24468 rows x 2 columns]


In [21]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [22]:
print(test)
print(test['image'])

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         images/test\angry\10052.jpg
1         images/test\angry\10065.jpg
2         images/test\angry\10079.jpg
3         images/test\angry\10095.jpg
4         images/test\angry\10121.jpg
                    ...              
7061    images/test\surprise\9806.jpg
7062    images/test\surprise\9830.jpg
7063    images/test\surprise\9853.jpg
7064    images/test\surprise\9878.jpg
7065     images/test\surprise\993.jpg
Name:

In [23]:
pip install Pillow


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [24]:
import PIL

In [25]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        # Load the image in grayscale mode
        img = load_img(image, color_mode='grayscale')
        img = img_to_array(img)  # Convert image to numpy array
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)  # Assuming images are 48x48
    return features

In [26]:
train_features = extract_features(train['image']) 


  0%|          | 0/24468 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [15]:
test_features = extract_features(test['image'])


  0%|          | 0/7066 [00:00<?, ?it/s]

In [18]:
x_train = train_features/255.0
x_test = test_features/255.0

In [16]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.Defaulting to user installation because normal site-packages is not writeable
     --------------------------------------- 11.1/11.1 MB 10.5 MB/s eta 0:00:00
     ---------------------------------------- 43.9/43.9 MB 5.5 MB/s eta 0:00:00
     -------------------------------------- 301.8/301.8 kB 6.2 MB/s eta 0:00:00




[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
from sklearn.preprocessing import LabelEncoder

In [28]:
le = LabelEncoder()
le.fit(train['label'])


LabelEncoder()

In [20]:
LabelEncoder()

LabelEncoder()

In [21]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])


In [22]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)


In [31]:
model = Sequential()


In [32]:
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

C:\Users\ipsit\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [27]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test)) 


Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 452s 2s/step - accuracy: 0.2416 - loss: 1.8336 - val_accuracy: 0.2583 - val_loss: 1.8105
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 445s 2s/step - accuracy: 0.2521 - loss: 1.8078 - val_accuracy: 0.2641 - val_loss: 1.8002
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 441s 2s/step - accuracy: 0.2626 - loss: 1.7701 - val_accuracy: 0.3208 - val_loss: 1.6633
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 338s 1s/step - accuracy: 0.3119 - loss: 1.6881 - val_accuracy: 0.4110 - val_loss: 1.5206
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 329s 1s/step - accuracy: 0.3824 - loss: 1.5634 - val_accuracy: 0.4604 - val_loss: 1.3999
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 315s 1s/step - accuracy: 0.4343 - loss: 1.4715 - val_accuracy: 0.4827 - val_loss: 1.3494
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 324s 1s/step - accuracy: 0.4480 - loss: 1.4206 - val_accuracy: 0.5045 - val_loss: 1.2985
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 329s 1s/step - accuracy: 0.4698 - loss: 1.3839 - 

In [35]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5") 

In [34]:
from keras.models import model_from_json


In [37]:
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [38]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']


In [39]:
def ef(image):
    img = load_img(image,color_mode = 'grayscale' )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0
    

In [40]:
image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

original image is of sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step
model prediction is  angry


In [1]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline